In [3]:
!pip uninstall -y torch torchtext torchaudio torchvision spacy thinc numpy sagemaker transformers
!pip install torch==2.1.0 torchtext==0.16.0 --force-reinstall --no-cache-dir
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir
!pip install spacy --no-cache-dir
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

# 6. Install other dependencies
!pip install sacrebleu --no-cache-dir
!pip install --upgrade --force-reinstall portalocker --no-cache-dir
!pip install sagemaker # Ensure sagemaker is compatible with numpy 1.x

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchtext 0.17.2
Uninstalling torchtext-0.17.2:
  Successfully uninstalled torchtext-0.17.2
Found existing installation: torchaudio 2.2.2
Uninstalling torchaudio-2.2.2:
  Successfully uninstalled torchaudio-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: sagemaker 2.247.0
Uninstalling sagemaker-2.247.0:
  Successfully uninstalled sagemaker-2.247.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 251.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 274.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 221.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import tarfile
import pickle
import os
import sys

sys.path.append('./my_transformer_code')
from torchtext.data.utils import get_tokenizer
from inference import load_model, translate_sentence
import sagemaker
from sagemaker.s3 import parse_s3_url


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
# --- 1. Configuration ---
# Paste the full S3 path to your trained model from the SageMaker job output
model_s3_path = "s3://spajjuri-transformers/transformer-model-v2-training-job/pytorch-training-2025-07-20-17-46-54-991/output/model.tar.gz"
# Define local names for the downloaded and extracted files
TAR_FILE_NAME = "model.tar.gz"
EXTRACT_DIR = "./temp_model_artefacts"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 2. Download the Model from S3 ---
print(f"Downloading model from {model_s3_path}...")
sess = sagemaker.Session()
bucket, key = parse_s3_url(model_s3_path)
sess.s3_client.download_file(bucket, key, TAR_FILE_NAME)
print(f" Model downloaded successfully as '{TAR_FILE_NAME}'")

# --- 3. Unpack the .tar.gz and Load Artefacts ---
print(f"Unpacking {TAR_FILE_NAME}...")
os.makedirs(EXTRACT_DIR, exist_ok=True)
with tarfile.open(TAR_FILE_NAME, "r:gz") as tar:
    tar.extractall(path=EXTRACT_DIR)

 Model downloaded successfully as 'model.tar.gz'
Unpacking model.tar.gz...


In [14]:
# Load vocabs
with open(os.path.join(EXTRACT_DIR, "en_vocab.pkl"), "rb") as f:
    en_vocab = pickle.load(f)
with open(os.path.join(EXTRACT_DIR, "de_vocab.pkl"), "rb") as f:
    de_vocab = pickle.load(f)
print("Vocabularies loaded.")

# Define path to the model weights file
model_path = os.path.join(EXTRACT_DIR, "best_transformer_model.pth")

# --- 4. Define Model Hyperparameters (must match the trained model) ---
embed_size = 512
num_layers = 6
forward_expansion = 4
heads = 8
dropout = 0.1
max_length = 256

# --- 5. Load the Model into Memory ---
model = load_model(
    en_vocab, de_vocab, embed_size, num_layers, forward_expansion,
    heads, dropout, DEVICE, max_length, model_path
)

print("Your model is now loaded and ready to use in the next cell!")

Vocabularies loaded.
Loading model...
Your model is now loaded and ready to use in the next cell!


In [15]:
from torchtext.data.utils import get_tokenizer

# Initialize the tokenizer
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

print("\n--- Interactive Translation Mode ---")
print("Type an English sentence and press Enter (type 'quit' to exit).")

while True:
    sentence = input("English: ")
    if sentence.lower() == 'quit':
        break
    
    translation = translate_sentence(
        model, sentence, en_vocab, de_vocab, en_tokenizer, DEVICE
    )
    
    print(f"German: {translation}")


--- Interactive Translation Mode ---
Type an English sentence and press Enter (type 'quit' to exit).


English:  Hello ! How are you


German: ['Laptops', 'Laptops', 'erörtern', 'Wolf', 'Wolf', 'Wolf', 'Wolf', 'Diner', 'Diner', 'Diner', 'Diner', 'Diner', 'Diner', 'Diner', 'Diner', 'Messbecher', 'Handgelenken', 'Handgelenken', 'Handgelenken', 'Handgelenken', 'Handgelenken', 'fasziniert', 'fasziniert', 'fasziniert', 'Handgelenken', 'blinde', 'blinde', 'blinde', 'Handgelenken', 'Handgelenken', 'Handgelenken', 'verzerrtes', 'verzerrtes', 'Steph', 'Steph', 'Steph', 'Steph', 'Steph', 'Steph', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'Ziegelsteinweg', 'verzerrtes']


English:  quit


In [ ]:
from torch.utils.data import DataLoader
from dataset import collate_fn # Assuming this is in your dataset.py
from inference import compute_bleu # Assuming this is in your inference.py

# You will need your test_data from the data loading step

# Create a DataLoader for the test set
test_dataloader = DataLoader(test_data, batch_size=32, collate_fn=lambda batch: collate_fn(batch, en_vocab, de_vocab))

# Calculate the BLEU score
score = compute_bleu(model, test_dataloader, en_vocab, de_vocab, en_tokenizer, en_tokenizer, DEVICE)

print(f"\nBLEU Score on the test set: {score*100:.2f}")